In [ ]:
import glob
import os
from pathlib import Path

print(os.getcwd())
os.chdir('../')
print(os.getcwd())

In [ ]:
import pandas as pd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.ChexpertLabeler.loader import Loader
from src.ChexpertLabeler.stages import Aggregator, Classifier, Extractor

In [ ]:
root_loc = './'
file_in = 'sample_reports.csv'

In [ ]:
loader = Loader(
    reports_path=Path(root_loc + file_in),
    # extract_impression=True,
)

loader.load()

In [ ]:
loader.collection.documents[0]

In [ ]:
# loader.collection

In [ ]:
extractor = Extractor(
    mention_phrases_dir=Path('./phrases/mention/'),
    unmention_phrases_dir=Path('./phrases/unmention/'),
    verbose=True,
)

In [ ]:
extractor.extract(loader.collection)

In [ ]:
classifier = Classifier(
    pre_negation_uncertainty_path='./patterns/pre_negation_uncertainty.txt',
    negation_path='./patterns/negation.txt',
    post_negation_uncertainty_path='./patterns/post_negation_uncertainty.txt',
    verbose=True
)

In [ ]:
classifier.classify(loader.collection)

In [ ]:
CATEGORIES = [
    "No Finding", 
    "Enlarged Cardiomediastinum", 
    "Cardiomegaly",
    "Lung Lesion", 
    "Lung Opacity", 
    "Edema", 
    "Consolidation",
    "Pneumonia", 
    "Atelectasis", 
    "Pneumothorax", 
    "Pleural Effusion",
    "Pleural Other", 
    "Fracture", 
    "Support Devices"
]

In [ ]:
aggregator = Aggregator(
    CATEGORIES,
    verbose=True
)

In [ ]:
labels = aggregator.aggregate(loader.collection)

In [ ]:
labels_sents = aggregator.aggregate_with_sentence(loader.collection)

In [ ]:
# long with doc, sentence, and label, able to group by sentence...

In [ ]:
labeled_reports = pd.DataFrame({"Reports": loader.reports})
for index, category in enumerate(CATEGORIES):
    labeled_reports[category] = labels[:, index]
    
labeled_reports

In [ ]:
%%time
df_long = pd.DataFrame(columns=['id', 'cat', 'vals', 'sents'])

for index, entry in enumerate(labels_sents):
    for ent, vals in entry.items():
        for val in vals:
            df_long = df_long.append(
                {
                    'id': index, 
                    'cat': ent, 
                    'vals': val[0],
                    'sents': val[1],
                }, ignore_index=True
            )

df_long.head()

In [ ]:
df_long.vals.value_counts()

In [ ]:
df_long.shape

In [ ]:
df_long['id'] = df_long['id'].astype(int)

In [ ]:
df_reports = pd.read_csv(root_loc + file_in, header=None).reset_index()

df_reports

In [ ]:
df_out = df_reports.merge(df_long, left_on='index', right_on='id', how='left')

df_out = df_out.drop(['id', 'index'], axis=1).rename(
    {
        0: 'mimic_id',
        1: 'report',
    }, axis=1
)

df_out

In [ ]:
df_out.to_csv('./data/mimic_processed/' + 'sents_' + file_in, index=False)